In [46]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [47]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-2f2bccd3-74e1-45c7-8fc6-b76cd59e8588'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [48]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [49]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11] and match_detail["info"]["gameMode"] == "CLASSIC":
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)

participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [50]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5833929889,1,sexhaver42,TOP,Aatrox,5/4/6,18,14845,32935,11056,...,4,14,1054,6630,3047,6333,3071,3044,3340,True
1,EUW1_5833929889,2,Chhen,JUNGLE,Ivern,3/6/12,15,5122,15834,5282,...,4,11,3107,2055,6617,1082,3158,3504,3364,True
2,EUW1_5833929889,3,Kevin Meas,MIDDLE,Talon,15/8/4,16,27429,26846,2638,...,4,14,6693,3142,3814,3158,6694,0,3364,True
3,EUW1_5833929889,4,Wolfsurfer,BOTTOM,Ezreal,12/3/6,16,26312,18522,157,...,3,4,4644,3024,3042,3158,6694,3508,3363,True
4,EUW1_5833929889,5,Amewtor,UTILITY,Zilean,2/4/18,15,6553,12715,5663,...,7,4,3853,4629,3158,3191,2420,2065,3364,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,EUW1_5826932804,6,MrangelXx,TOP,Gragas,7/5/9,18,13668,35991,14463,...,4,12,3121,6664,3111,4401,3076,1011,3363,False
176,EUW1_5826932804,7,BDM ShΔωtΨ,JUNGLE,JarvanIV,8/3/20,16,19950,44826,15704,...,4,11,6664,3157,8001,3111,8020,3110,3364,False
177,EUW1_5826932804,8,BDM Noggy,MIDDLE,Vladimir,13/7/7,17,28564,37401,26575,...,4,14,3152,3157,4629,3065,3020,1058,3364,False
178,EUW1_5826932804,9,DaxOlessa,BOTTOM,Caitlyn,5/14/9,16,21166,28111,2276,...,4,7,6671,3006,3031,3094,1053,1038,3363,False


In [51]:
matchesDf

,matchId,gameDuration,gameType,gameVersion
0,EUW1_5833929889,1803,MATCHED_GAME,12.7.433.4138
1,EUW1_5833962481,1956,MATCHED_GAME,12.7.433.4138
2,EUW1_5833827508,1465,MATCHED_GAME,12.7.433.4138
3,EUW1_5833770214,1975,MATCHED_GAME,12.7.433.4138
4,EUW1_5833687517,1286,MATCHED_GAME,12.7.433.4138
5,EUW1_5833733388,914,MATCHED_GAME,12.7.433.4138
6,EUW1_5832387212,1984,MATCHED_GAME,12.7.433.4138
7,EUW1_5832361338,1811,MATCHED_GAME,12.7.433.4138
8,EUW1_5829524931,1867,MATCHED_GAME,12.7.433.4138
9,EUW1_5829408711,1810,MATCHED_GAME,12.7.433.4138


In [52]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [53]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [54]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [55]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5833929889,ITEM_PURCHASED,9.0,1036.0,2749,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
1,EUW1_5833929889,ITEM_PURCHASED,9.0,2003.0,3112,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
2,EUW1_5833929889,ITEM_PURCHASED,9.0,2003.0,3277,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
3,EUW1_5833929889,ITEM_PURCHASED,9.0,2003.0,3409,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
4,EUW1_5833929889,ITEM_PURCHASED,2.0,1035.0,3772,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Chhen,Ivern,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11965,EUW1_5826932804,ITEM_PURCHASED,10.0,3157.0,2180526,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36.0,Okay Motomami,Lux,NaN,NaN,NaN,NaN,False
11966,EUW1_5826932804,BUILDING_KILL,NaN,NaN,2184193,NaN,NaN,NaN,1.0,NaN,...,INHIBITOR_BUILDING,MID_LANE,36.0,NaN,NaN,tankmoplayerop,Teemo,NaN,NaN,False
11967,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,2187827,NaN,NaN,NaN,4.0,9.0,...,NaN,NaN,36.0,NaN,NaN,Kelody,Twitch,DaxOlessa,Caitlyn,False
11968,EUW1_5826932804,BUILDING_KILL,NaN,NaN,2193474,NaN,NaN,NaN,4.0,NaN,...,TOWER_BUILDING,MID_LANE,37.0,NaN,NaN,Kelody,Twitch,NaN,NaN,False


In [56]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
8,EUW1_5833929889,9
18,EUW1_5833962481,9
28,EUW1_5833827508,9
33,EUW1_5833770214,4
48,EUW1_5833687517,9
58,EUW1_5833733388,9
63,EUW1_5832387212,4
78,EUW1_5832361338,9
87,EUW1_5829524931,8
92,EUW1_5829408711,3


In [57]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5833929889,ITEM_PURCHASED,9.0,1036.0,2749,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
1,EUW1_5833929889,ITEM_PURCHASED,9.0,2003.0,3112,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
2,EUW1_5833929889,ITEM_PURCHASED,9.0,2003.0,3277,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
3,EUW1_5833929889,ITEM_PURCHASED,9.0,2003.0,3409,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
4,EUW1_5833929889,LEVEL_UP,9.0,NaN,144296,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,2.0,Kelody,Twitch,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,2174346,NaN,NaN,NaN,4.0,7.0,...,NaN,NaN,36.0,NaN,NaN,Kelody,Twitch,BDM ShΔωtΨ,JarvanIV,False
1271,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,2175601,NaN,NaN,NaN,4.0,6.0,...,NaN,NaN,36.0,NaN,NaN,Kelody,Twitch,MrangelXx,Gragas,False
1272,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,2187827,NaN,NaN,NaN,4.0,9.0,...,NaN,NaN,36.0,NaN,NaN,Kelody,Twitch,DaxOlessa,Caitlyn,False
1273,EUW1_5826932804,BUILDING_KILL,NaN,NaN,2193474,NaN,NaN,NaN,4.0,NaN,...,TOWER_BUILDING,MID_LANE,37.0,NaN,NaN,Kelody,Twitch,NaN,NaN,False


In [58]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == True]

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
34,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1163141,NaN,NaN,NaN,6.0,1.0,...,NaN,NaN,19.0,NaN,NaN,My Little Flori,Gangplank,sexhaver42,Aatrox,True
40,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1249943,NaN,NaN,NaN,8.0,5.0,...,NaN,NaN,21.0,NaN,NaN,Warden of Helia,Viktor,Amewtor,Zilean,True
49,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1415202,NaN,NaN,NaN,6.0,2.0,...,NaN,NaN,24.0,NaN,NaN,My Little Flori,Gangplank,Chhen,Ivern,True
50,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1415202,NaN,NaN,NaN,6.0,4.0,...,NaN,NaN,24.0,NaN,NaN,My Little Flori,Gangplank,Wolfsurfer,Ezreal,True
51,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1416756,NaN,NaN,NaN,6.0,1.0,...,NaN,NaN,24.0,NaN,NaN,My Little Flori,Gangplank,sexhaver42,Aatrox,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,1186667,NaN,NaN,NaN,3.0,9.0,...,NaN,NaN,20.0,NaN,NaN,Hallgren,Karthus,DaxOlessa,Caitlyn,True
1234,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,1605570,NaN,NaN,NaN,2.0,8.0,...,NaN,NaN,27.0,NaN,NaN,Cency,Amumu,BDM Noggy,Vladimir,True
1246,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,1821654,NaN,NaN,NaN,3.0,9.0,...,NaN,NaN,30.0,NaN,NaN,Hallgren,Karthus,DaxOlessa,Caitlyn,True
1260,EUW1_5826932804,ELITE_MONSTER_KILL,NaN,NaN,1971923,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,33.0,NaN,NaN,Cency,Amumu,NaN,NaN,True


Check how often a Player died in Early Game (pre 10 mins)

In [59]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
8,EUW1_5833929889,9,1
18,EUW1_5833962481,9,1
28,EUW1_5833827508,9,3
33,EUW1_5833770214,4,3
48,EUW1_5833687517,9,3
58,EUW1_5833733388,9,3
63,EUW1_5832387212,4,5
78,EUW1_5832361338,9,2
87,EUW1_5829524931,8,1
92,EUW1_5829408711,3,1


In [60]:
#get CS per minute
def getCSperMinute(row, name):
    gameDuration = matchesDf.loc[matchesDf["matchId"] == row["matchId"]]["gameDuration"].values[0]/60
    csPerGame = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["totalMinionsKilled"].values[0]
    return csPerGame / gameDuration

In [74]:
csPerGame = participantsDf.loc[(participantsDf["matchId"] == "EUW1_5833929889") & (participantsDf["summonerName"] == "Kelody")]["totalMinionsKilled"].values[0]

csPerGame

129

In [61]:
kelody["csPerMinute"] = kelody.apply(lambda row: getCSperMinute(row, "Kelody"), axis = 1)

In [62]:
kelody

,matchId,participant,deathsPreMin10,csPerMinute
8,EUW1_5833929889,9,1,0.071547
18,EUW1_5833962481,9,1,0.069530
28,EUW1_5833827508,9,3,0.081911
33,EUW1_5833770214,4,3,0.069873
48,EUW1_5833687517,9,3,0.095645
58,EUW1_5833733388,9,3,0.108315
63,EUW1_5832387212,4,5,0.074597
78,EUW1_5832361338,9,2,0.089453
87,EUW1_5829524931,8,1,0.080343
92,EUW1_5829408711,3,1,0.062431


#### TODO
##### Features:
- <s>Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)</s>

- Died with ... Gold

- <s>count 1v1 kill/deaths</s>  
  
- <s>Good roaming (assists pre 15 min)</s>

- <s>Warding (check ward placements pre 15 min) maybe in combination with ganks</s>

- CS per min (if data available)

- <s>KDA (kills and assists divided by deaths) als Output KDA String aus participantDf</s>

- <s>Tower Kills/Assists</s> 

- <s>Objective Kills/Assists (Baron, Dragon, Herald)</s> 



##### Features on the tableau visualization:

- <s>Timeline</s>

- <s>Champion Icons</s>

- <s>Damage Dealt Bar Chart (participantDf)</s>

- Game win (participantDf)



Check 1v1

In [63]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
33,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1154847,NaN,NaN,NaN,1.0,9.0,...,NaN,19.0,NaN,NaN,sexhaver42,Aatrox,Kelody,Twitch,False,True
63,EUW1_5833929889,CHAMPION_KILL,NaN,NaN,1658643,NaN,NaN,NaN,3.0,9.0,...,NaN,28.0,NaN,NaN,Kevin Meas,Talon,Kelody,Twitch,False,True
121,EUW1_5833962481,CHAMPION_KILL,NaN,NaN,1656926,NaN,NaN,NaN,3.0,9.0,...,NaN,28.0,NaN,NaN,Authority of Gød,Yasuo,Kelody,Twitch,False,True
135,EUW1_5833962481,CHAMPION_KILL,NaN,NaN,1950856,NaN,NaN,NaN,2.0,9.0,...,NaN,33.0,NaN,NaN,Drıft Kıng,Viego,Kelody,Twitch,False,True
160,EUW1_5833827508,CHAMPION_KILL,NaN,NaN,765465,NaN,NaN,NaN,4.0,9.0,...,NaN,13.0,NaN,NaN,cuppcakee,Samira,Kelody,Twitch,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,EUW1_5826907936,CHAMPION_KILL,NaN,NaN,746707,NaN,NaN,NaN,1.0,9.0,...,NaN,12.0,NaN,NaN,KFC CEO,Fiora,Kelody,Jinx,False,True
1143,EUW1_5826907936,CHAMPION_KILL,NaN,NaN,983894,NaN,NaN,NaN,1.0,9.0,...,NaN,16.0,NaN,NaN,KFC CEO,Fiora,Kelody,Jinx,False,True
1146,EUW1_5826907936,CHAMPION_KILL,NaN,NaN,1037215,NaN,NaN,NaN,4.0,9.0,...,NaN,17.0,NaN,NaN,iSlach,Xayah,Kelody,Jinx,False,True
1240,EUW1_5826932804,CHAMPION_KILL,NaN,NaN,1681447,NaN,NaN,NaN,4.0,10.0,...,NaN,28.0,NaN,NaN,Kelody,Twitch,Okay Motomami,Lux,False,True


In [64]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)

In [65]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name) & (kelodyEventsDf["1v1"] == False)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)

In [66]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)

In [67]:
def deathToGank(row, name):
    if row["1v1"] == False and row["type"] == "CHAMPION_KILL" and row["victimName"] == name: 
        enemyJungler = participantsDf.loc[(row["matchId"] == participantsDf["matchId"]) & (participantsDf["teamPosition"] == "JUNGLE")]["summonerName"].values[0]
        if row["killerName"] == enemyJungler:
            return True
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == row["matchId"])]["teamPosition"].values[0] == "JUNGLE":
                return True
    return False
kelodyEventsDf["deathToGank"] = kelodyEventsDf.apply(lambda row: deathToGank(row, "Kelody"), axis=1)

In [68]:
def countDeathsToGanks(row, name):
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["deathToGank"] == True)
            & (kelodyEventsDf["victimName"] == name)
            & (kelodyEventsDf["timestampMIN"] <= 15)
        ]
    )


kelody["deathsToGanks"] = kelody.apply(
    lambda row: countDeathsToGanks(row, "Kelody"), axis=1
)


In [69]:
def countWardsPlaced(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["participant"].values[0]
    return len(
        kelodyEventsDf[
            (kelodyEventsDf["matchId"] == row["matchId"])
            & (kelodyEventsDf["creatorId"] == participantId)
            & (kelodyEventsDf["type"] == "WARD_PLACED")
        ]
    )


kelody["wardsPlaced"] = kelody.apply(
    lambda row: countWardsPlaced(row, "Kelody"), axis=1
)


In [70]:
def countWardsPlacedBeforeGanked(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (
        participantsDf["summonerName"] == name)]["participant"].values[0]
    wardsPlacedTimestamps = kelodyEventsDf.loc[(kelodyEventsDf["matchId"] == row["matchId"]) & (
        kelodyEventsDf["creatorId"] == participantId) & (kelodyEventsDf["type"] == "WARD_PLACED")]["timestampMIN"].values
    ganksTimestamps = kelodyEventsDf.loc[
        (kelodyEventsDf["matchId"] == row["matchId"])
        & (kelodyEventsDf["deathToGank"] == True)
        & (kelodyEventsDf["victimName"] == name)
        & (kelodyEventsDf["timestampMIN"] <= 15)
    ]["timestampMIN"].values
    wardsPlacedBeforeGank = 0
    for timestamp in ganksTimestamps:
        for wardTimestamp in wardsPlacedTimestamps:
            if(timestamp - wardTimestamp <= 2 and timestamp - wardTimestamp > 0):
                wardsPlacedBeforeGank += 1
    return wardsPlacedBeforeGank

In [71]:
kelody["wardsPlacedBeforeGanks"] = kelody.apply(lambda row: countWardsPlacedBeforeGanked(row, "Kelody"), axis=1)

In [72]:
def checkRoamKills(row, name):
    if row["assistingParticipantIds"] and (row["killerName"] == name) and (row["type"] == "CHAMPION_KILL") and (row["timestampMIN"] <= 15):
        return True
    return False
-
def countRoamKills(row):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & ((kelodyEventsDf["kelodyAssist"] == True) & (kelodyEventsDf["timestampMIN"] <= 15) | (kelodyEventsDf["kelodyRoamKill"] == True))])

SyntaxError: invalid syntax (2034829799.py, line 5)

In [ ]:
kelodyEventsDf["kelodyRoamKill"] = kelodyEventsDf.apply(lambda row: checkRoamKills(row, "Kelody"), axis = 1)
kelody["roamKillsAssists"] = kelody.apply(lambda row: countRoamKills(row), axis=1)

In [ ]:
def gameWin(row, name):
    return participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["win"].values[0]

In [ ]:
kelody["gameWin"] = kelody.apply(lambda row: gameWin(row, "Kelody"), axis=1)

In [ ]:
def getKDA(row, name):
    kdaString = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["kda"].values[0]
    kdaStringArr = kdaString.split("/")
    kdaArr = [int(kdaArrPart) for kdaArrPart in kdaStringArr]
    try:
        kda = (kdaArr[0]+kdaArr[2]) / kdaArr[1]
    except:
        kda = (kdaArr[0]+kdaArr[2]) / 1
    return round(kda, 2)

In [ ]:
kelody["kda"] = kelody.apply(lambda row: getKDA(row, "Kelody"), axis=1)

In [ ]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced,wardsPlacedBeforeGanks,roamKillsAssists,gameWin,kda
7,EUW1_5829524931,8,1,5,1,1,0,0,0,4,14,4,10,False,1.90
12,EUW1_5829408711,3,1,5,3,0,0,0,0,1,13,0,6,False,1.17
27,EUW1_5829375821,8,0,1,0,1,0,0,1,0,4,0,1,True,1.00
36,EUW1_5827639869,7,2,1,1,0,1,0,0,2,3,1,4,False,1.20
46,EUW1_5827587343,7,2,1,0,0,1,0,0,3,5,2,9,False,1.67
58,EUW1_5827621876,9,3,1,0,1,0,0,0,2,13,2,3,False,1.88
67,EUW1_5827066877,8,4,0,8,0,2,1,0,0,14,0,0,False,1.25
77,EUW1_5827020811,8,2,6,6,0,0,0,0,2,19,0,0,False,0.78
88,EUW1_5826907936,9,4,1,3,0,0,0,0,1,8,2,6,False,1.10
93,EUW1_5826932804,4,2,2,0,4,0,1,0,1,16,0,9,True,2.45


In [ ]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
